### hr 과 lr 을 new_train, new_valid 로 분리

In [31]:
from glob import glob
import os
import shutil
import random

### for, ubuntu
base_paths = ['/home/jovyan/playGround/ziippy/dl_data/open_sr/train/hr/', 
             '/home/jovyan/playGround/ziippy/dl_data/open_sr/train/lr/']
new_train_paths = ['/home/jovyan/playGround/ziippy/dl_data/open_sr/new_train_hr/hr_split_train/',
                  '/home/jovyan/playGround/ziippy/dl_data/open_sr/new_train_lr/lr_split_train/']
new_valid_paths = ['/home/jovyan/playGround/ziippy/dl_data/open_sr/new_valid/hr_split_valid/',
                  '/home/jovyan/playGround/ziippy/dl_data/open_sr/new_valid/lr_split_valid/']

random.seed(100)

valid_count = 40
valid_index_list = []

for index, base_path in enumerate(base_paths):
    new_train_path = new_train_paths[index]
    new_valid_path = new_valid_paths[index]
    
    print(new_train_path)
    print(new_valid_path)
    
    shutil.rmtree(new_train_path, ignore_errors=True)
    os.makedirs(new_train_path, exist_ok=True)
    shutil.rmtree(new_valid_path, ignore_errors=True)
    os.makedirs(new_valid_path, exist_ok=True)

    file_list = glob(base_path + '*.png')
    file_count = len(file_list)
    print(file_count)
    
    file_list.sort()
    
    if len(valid_index_list) == 0:
        # get valid index from random
        for _ in range(valid_count):
            valid_index_list.append(random.randint(0, file_count))
        valid_index_list.sort()
        print(valid_index_list)

    for index, file in enumerate(file_list):
        filename = file.split('/')[-1]
        if index not in valid_index_list:
            shutil.copy(file, new_train_path+filename)
        else:
            shutil.copy(file, new_valid_path+filename)
            print(filename + ' > val')
        # print(filename)
    
print('all done')

/home/jovyan/playGround/ziippy/dl_data/open_sr/new_train_hr/hr_split_train/
/home/jovyan/playGround/ziippy/dl_data/open_sr/new_valid/hr_split_valid/
1640
[98, 164, 225, 248, 288, 298, 357, 365, 386, 417, 418, 431, 470, 539, 564, 633, 686, 710, 716, 757, 804, 825, 839, 887, 931, 933, 941, 944, 1038, 1091, 1137, 1282, 1326, 1349, 1444, 1499, 1508, 1572, 1578, 1638]
open_0098.png > val
open_0164.png > val
open_0225.png > val
open_0248.png > val
open_0288.png > val
open_0298.png > val
open_0357.png > val
open_0365.png > val
open_0386.png > val
open_0417.png > val
open_0418.png > val
open_0431.png > val
open_0470.png > val
open_0539.png > val
open_0564.png > val
open_0633.png > val
open_0686.png > val
open_0710.png > val
open_0716.png > val
open_0757.png > val
open_0804.png > val
open_0825.png > val
open_0839.png > val
open_0887.png > val
open_0931.png > val
open_0933.png > val
open_0941.png > val
open_0944.png > val
open_1038.png > val
open_1091.png > val
open_1137.png > val
open_1282.png 

### Adjust transform to train

In [64]:
import os, shutil
from PIL import Image
from glob import glob
from tqdm import tqdm

import numpy as np
import cv2

### for, ubuntu - open data
# hr_path = '/home/jovyan/playGround/ziippy/dl_data/open_sr/new_train_hr/hr_split_train/'
# new_hr_path = '/home/jovyan/playGround/ziippy/dl_data/open_sr/new_train_hr/hr_split_train_aug/'
# new_lr_path = '/home/jovyan/playGround/ziippy/dl_data/open_sr/new_train_lr/lr_split_train_aug/'
# new_valid_hr_path = '/home/jovyan/playGround/ziippy/dl_data/open_sr/new_valid/hr_split_valid_aug/'
# new_valid_lr_path = '/home/jovyan/playGround/ziippy/dl_data/open_sr/new_valid/lr_split_valid_aug/'
# FILE_PREFIX = 'AUG_'
# validation_count = 20
#random.seed(200)

### for, ubuntu - my data
hr_path = '/home/jovyan/playGround/ziippy/dl_data/my_sr/hr/'
new_hr_path = '/home/jovyan/playGround/ziippy/dl_data/open_sr/new_train_hr/my_hr_train/'
new_lr_path = '/home/jovyan/playGround/ziippy/dl_data/open_sr/new_train_lr/my_lr_train/'
new_valid_hr_path = '/home/jovyan/playGround/ziippy/dl_data/open_sr/new_valid/my_hr_valid/'
new_valid_lr_path = '/home/jovyan/playGround/ziippy/dl_data/open_sr/new_valid/my_lr_valid/'
FILE_PREFIX = 'MY_'
validation_count = 20
random.seed(17)

path_splitter = '/'

os.makedirs(new_hr_path, exist_ok=True)
os.makedirs(new_lr_path, exist_ok=True)
os.makedirs(new_valid_hr_path, exist_ok=True)
os.makedirs(new_valid_lr_path, exist_ok=True)

# remove old files
filelist = glob(os.path.join(new_hr_path, "*"))
for f in filelist:
    os.remove(f)
filelist = glob(os.path.join(new_lr_path, "*"))
for f in filelist:
    os.remove(f)
filelist = glob(os.path.join(new_valid_hr_path, "*"))
for f in filelist:
    os.remove(f)
filelist = glob(os.path.join(new_valid_lr_path, "*"))
for f in filelist:
    os.remove(f)
print('remove old - done')

# variables
target_width = 512
target_height = 512

do_each = False # if true then generate all mode

total_count = 0
current_count = 0
current_valid_count = 0
   
# hr
filelist = glob(os.path.join(hr_path, "*.png"))
filelist.sort()

total_count = len(filelist)
valid_index_list = []
# get valid index from random
for _ in range(validation_count):
    valid_index_list.append(random.randint(0, total_count))
valid_index_list.sort()
print(valid_index_list)

### resize or filter
modes = ['nearest', 'box', 'bilinear', 'hamming', 'bicubic', 'lanczos', 'blur', 'gaussian_blur', 'median_blur', 'bilateral_filter']
# applys = [True, False, True, False, True, True, False, True, False, True]
applys = [True, True, True, True, True, True, True, True, True, True]
            
for _, file in enumerate(tqdm(filelist)):
    filename = file.split(path_splitter)[-1]
    filename_without_ext = filename.split('.')[0]
    
    do_modes = []
    for index, mode in enumerate(modes):
        if applys[index] is True:
            do_modes.append(mode)
    
    if do_each is False: # 파일마다 한번씩만 round robin
        if current_count in valid_index_list:
            one_mode = do_modes[current_valid_count % len(do_modes)]
        else:
            one_mode = do_modes[current_count % len(do_modes)]
        do_modes.clear()
        do_modes.append(one_mode)
    
    target_hr_path = new_hr_path
    target_lr_path = new_lr_path
    if current_count in valid_index_list:
        target_hr_path = new_valid_hr_path
        target_lr_path = new_valid_lr_path
        current_valid_count += 1
    
    for mode in do_modes:
        if mode in ['nearest', 'box', 'bilinear', 'hamming', 'bicubic', 'lanczos']:
            # resize
            img = Image.open(file)
            if mode == 'nearest':
                img_resize = img.resize((target_width, target_height), Image.NEAREST)
            elif mode == 'box':
                img_resize = img.resize((target_width, target_height), Image.BOX)
            elif mode == 'bilinear':
                img_resize = img.resize((target_width, target_height), Image.BILINEAR)
            elif mode == 'hamming':
                img_resize = img.resize((target_width, target_height), Image.HAMMING)
            elif mode == 'bicubic':
                img_resize = img.resize((target_width, target_height), Image.BICUBIC)
            elif mode == 'lanczos':
                img_resize = img.resize((target_width, target_height), Image.LANCZOS)
                
            # copy hr
            new_file_hr = target_hr_path + FILE_PREFIX + filename_without_ext + "_" + mode + ".png"
            #shutil.copy(file, new_file_hr)
            os.symlink(file, new_file_hr)
            # copy lr
            new_file_lr = target_lr_path + FILE_PREFIX + filename_without_ext + "_" + mode + ".png"
            img_resize.save(new_file_lr)
        
            #print(filename + ' ' + mode + ' done')
        else:
            img = cv2.imread(file)
            resize_img = cv2.resize(img, (target_width, target_height))   
            if mode == 'blur':
                dst = cv2.blur(resize_img, (5,5))
            elif mode == 'gaussian_blur':
                dst = cv2.GaussianBlur(resize_img, (5,5), 0)
            elif mode == 'median_blur':
                dst = cv2.medianBlur(resize_img, 5)
            elif mode == 'bilateral_filter':
                dst = cv2.bilateralFilter(resize_img, 9, 75, 75)
            else:
                continue
            
            # copy hr
            new_file_hr = target_hr_path + FILE_PREFIX + filename_without_ext + "_" + mode + ".png"
            #shutil.copy(file, new_file_hr)
            os.symlink(file, new_file_hr)
            # copy lr
            new_file_lr = target_lr_path + FILE_PREFIX + filename_without_ext + "_" + mode + ".png"
            cv2.imwrite(new_file_lr, dst)

            #print(filename + ' ' + mode + ' done')
    
    current_count += 1
    #if current_count > 10:
    #    break
    
print('all done')

### 컨볼루션
#img = cv2.imread(hr_path + filename)
#resize_img = cv2.resize(img, (target_width, target_height))
#kernel = np.ones((5,5),np.float32)/25
#dst = cv2.filter2D(resize_img, -1, kernel)
#new_file = lr_path + '7_convolution_' + filename
#cv2.imwrite(new_file, dst)

remove old - done
[27, 112, 140, 178, 254, 258, 284, 296, 310, 325, 374, 393, 411, 424, 429, 513, 534, 553, 564, 655]


100%|██████████| 660/660 [01:54<00:00,  5.74it/s]

all done


## CSV 생성

In [54]:
import csv

hr_path = '/home/jovyan/playGround/ziippy/dl_data/open_sr/new_train_hr/'
lr_path = '/home/jovyan/playGround/ziippy/dl_data/open_sr/new_train_lr/'
csv_file = '/home/jovyan/playGround/ziippy/dl_data/open_sr/new_train.csv'

HEAD = ['LR', 'HR']

def scanRecurse(baseDir):
    for entry in os.scandir(baseDir):
        if entry.is_file():
            yield entry.path
        elif entry.is_dir():
            if '.' in entry.name:
                pass
            else:
                yield from scanRecurse(entry.path)
            
lr_all_list = []
for file in scanRecurse(lr_path):
    lr_all_list.append(file)
lr_all_list.sort()
print(len(lr_all_list))
print(lr_all_list[0])
print(lr_all_list[-1])

hr_all_list = []
for file in scanRecurse(hr_path):
    hr_all_list.append(file)
hr_all_list.sort()
print(len(hr_all_list))
print(hr_all_list[0])
print(hr_all_list[-1])

train_list = []
for index, lr_file in enumerate(hr_all_list):
    train = [lr_file, hr_all_list[index]]
    train_list.append(train)
    
#print(train_list)

with open(csv_file,'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(HEAD)
    writer.writerows(train_list)
f.close()

3840
/home/jovyan/playGround/ziippy/dl_data/open_sr/new_train_lr/lr_split_train/open_0000.png
/home/jovyan/playGround/ziippy/dl_data/open_sr/new_train_lr/my_lr_train/MY_IMG_20220927_122951_4_CE_bilateral_filter.png
3840
/home/jovyan/playGround/ziippy/dl_data/open_sr/new_train_hr/hr_split_train/open_0000.png
/home/jovyan/playGround/ziippy/dl_data/open_sr/new_train_hr/my_hr_train/MY_IMG_20220927_122951_4_CE_bilateral_filter.png
